In [3]:
from rdflib import Graph, Namespace, Literal, RDF, URIRef
from rdflib.namespace import FOAF, DC, XSD

# Définition des namespaces
EX = Namespace("http://example.org/")
SCHEMA = Namespace("http://schema.org/")
MO = Namespace("http://purl.org/ontology/mo/")

# Création du graphe
g = Graph()
g.bind("foaf", FOAF)
g.bind("dc", DC)
g.bind("schema", SCHEMA)
g.bind("mo", MO)
g.bind("ex", EX)

In [7]:
import musicbrainzngs

musicbrainzngs.set_useragent("musicbrainzngs","0.7.0")

In [ ]:
import requests
import time

HEADERS = {
    "User-Agent": "RapCollabParser/1.0 ( contact@example.org )"
}

BASE_URL = "https://musicbrainz.org/ws/2"

def search_artist(name):
    url = f"{BASE_URL}/artist"
    params = {
        "query": name,
        "fmt": "json",
        "limit": 1
    }
    response = requests.get(url, params=params, headers=HEADERS)
    response.raise_for_status()
    results = response.json()
    if results["artists"]:
        return results["artists"][0]
    return None

def get_artist_releases(mbid):
    url = f"{BASE_URL}/release"
    params = {
        "artist": mbid,
        "inc": "release-groups",
        "fmt": "json",
        "limit": 100
    }
    response = requests.get(url, params=params, headers=HEADERS)
    response.raise_for_status()
    return response.json()["releases"]

def get_artist_details(mbid):
    url = f"{BASE_URL}/artist/{mbid}"
    params = {
        "inc": "area+begin-area+aliases",
        "fmt": "json"
    }
    response = requests.get(url, params=params, headers=HEADERS)
    time.sleep(1)  # Respecter les limites de l'API
    response.raise_for_status()
    return response.json()

# Exemple d'utilisation
if __name__ == "__main__":
    name = "Nekfeu"
    artist = search_artist(name)

    if artist:
        mbid = artist["id"]
        print(f"🎤 Artiste trouvé : {artist['name']} (MBID: {mbid})")

        details = get_artist_details(mbid)
        time.sleep(1)  # Respecter les limites de l'API
        releases = get_artist_releases(mbid)

        print("Nom complet :", details.get("name"))
        print("Alias :", [alias["name"] for alias in details.get("aliases", [])])
        print("Pays :", details.get("area", {}).get("name"))
        print("Lieu de naissance :", details.get("begin-area", {}).get("name"))
        print("Date de naissance :", details.get("life-span", {}).get("begin"))

        print("\n🎶 Albums / Releases :")
        for r in releases[:10]:  # pour limiter l'affichage
            print("-", r["title"], "(date:", r.get("date", "n/a") + ")")

    else:
        print("❌ Aucun artiste trouvé.")


🎤 Artiste trouvé : Nekfeu (MBID: d721866d-5640-44ee-87f7-23dd062abd8a)


HTTPError: 400 Client Error: Bad Request for url: https://musicbrainz.org/ws/2/artist/d721866d-5640-44ee-87f7-23dd062abd8a?inc=area%2Bbegin-area%2Baliases&fmt=json

In [5]:
# === Construction RDF

# Artistes
for artist in artists:
    artist_uri = EX[artist["id"]]
    g.add((artist_uri, RDF.type, MO.MusicArtist))
    g.add((artist_uri, FOAF.name, Literal(artist["stage_name"])))
    g.add((artist_uri, FOAF.givenName, Literal(artist["full_name"].split()[0])))
    g.add((artist_uri, FOAF.familyName, Literal(artist["full_name"].split()[-1])))
    g.add((artist_uri, SCHEMA.birthPlace, Literal(artist["birthplace"])))
    g.add((artist_uri, SCHEMA.birthDate, Literal(artist["birthdate"], datatype=XSD.date)))

    for album_id in artist["albums"]:
        g.add((artist_uri, MO.produced, EX[album_id]))
        g.add((artist_uri, MO.published_album, EX[album_id]))

    for song_id in artist["songs"]:
        g.add((artist_uri, MO.produced, EX[song_id]))

    for feat_id in artist["features"]:
        g.add((artist_uri, MO.featured_on, EX[feat_id]))

# Musiques
for song_id, song in songs.items():
    song_uri = EX[song_id]
    g.add((song_uri, RDF.type, MO.MusicalWork))
    g.add((song_uri, DC.title, Literal(song["title"])))
    g.add((song_uri, MO.performer, EX[song["author"]]))
    g.add((song_uri, DC.date, Literal(song["release_date"], datatype=XSD.date)))

# Albums
for album_id, album in albums.items():
    album_uri = EX[album_id]
    g.add((album_uri, RDF.type, MO.Release))
    g.add((album_uri, DC.title, Literal(album["title"])))
    g.add((album_uri, MO.producer, EX[album["author"]]))
    g.add((album_uri, DC.date, Literal(album["release_date"], datatype=XSD.date)))

# === Export en Turtle
g.serialize("music_graph.ttl", format="turtle")
print("✅ RDF exporté dans 'music_graph.ttl'")


✅ RDF exporté dans 'music_graph.ttl'
